In [1]:
import shapefile
from json import dumps
import json
from pathlib import Path

In [2]:
path = Path.cwd()/'main'/'Lands' # the path to work with

In [3]:
def shp_to_geo(path: Path) -> None:
    """ Gets a path of shp data file and write the geojson data in the same path
    
    Parameters
    ----------
    path: pathlib.Path
        path of .shp of each land
    
    Returns
    ---------
    None
        Outputs to the geo.json in the same path
        """    
    reader = shapefile.Reader(path.as_posix(), encoding="UTF-8")
    fields = reader.fields[1:]
    field_names = [field[0] for field in fields]

    buffer = []
    i = 1
    for sr in reader.shapeRecords():
        atr = dict(zip(field_names, sr.record))
        geom = sr.shape.__geo_interface__
        buffer.append(dict(type="Feature", geometry=geom, properties=atr))
        i += 1

    # write the GeoJSON file
    geojson = open(path.parent.as_posix()+'/geo.json', "w")
    geojson.write(dumps({"type": "FeatureCollection", "features": buffer}, indent=2) + "\n")
    geojson.close()

In [4]:
def json_to_poly(path: Path) -> None:
    """ Gets a path of geojson data file and write the polygon data in the same path
    
    Parameters
    ----------
    path: pathlib.Path
        path of geo.json of .shp in the same path of .shp
    
    Returns
    ---------
    None
        Outputs to the polygon.json in the same path
        """
    final_list = []
    cor_list = []
    json_path = open(path.parent.as_posix()+'/geo.json')
    json_file = json.load(json_path)

    for item in json_file['features']:
        for index in item['geometry']['coordinates'][0]:
            cor_list.append([index[1], index[0]])
            print(cor_list)
        coordinate = str([cor_list])
        print(coordinate)
        polygon = 'POLYGON(({}))'.format(
            coordinate.replace('[', '').replace(',', '').replace('] ', ',').replace(']]]', ''))
        final_list.append(polygon)
        print(final_list)
        cor_list = []

    geojson = open(path.parent.as_posix()+'/polygon.json',"w")
    geojson.write(json.dumps({"polygons": final_list}, indent=2) + "\n")
    geojson.close()

In [5]:
def shp_to_geo_and_poly(): # Convert every .shp file to geo.json and poly.json in the same path
    sh_paths = [p for p in path.glob('**/*.shp')]
    for p in sh_paths:
        shp_to_geo(p)
        json_to_poly(p)

In [6]:
shp_to_geo_and_poly() # Run the function and Do the job!